### Import packages

In [3]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Create model

In [25]:
# Define model
m = AbstractModel()

# Define sets
m.ITEMS = Set()
m.PGRID = Set()

# Define Params
m.mrkPriceGrid = Param(m.ITEMS, m.PGRID)
m.demandPriceGrid = Param(m.ITEMS, m.PGRID)
m.revenuePriceGrid = Param(m.ITEMS, m.PGRID)
m.basePrice = Param(m.ITEMS, initialize = {'A':21.5, 'B':29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10, 'B':30, 'C':41})
m.maxPct = 0.80
m.cap = 20
m.dispCost = 1.2

# Define Decision Variable
m.SelectPrice = Var(m.ITEMS, m.PGRID, domain = Binary)

# Define Implicit Variables
m.MarkPct = Var(m.ITEMS, domain = NonNegativeReals)
m.MarkPrice = Var(m.ITEMS, domain = NonNegativeReals)
m.Demand = Var(m.ITEMS, domain = NonNegativeReals)
m.Revenue = Var(m.ITEMS, domain = NonNegativeReals)
m.FinalInv = Var(m.ITEMS, domain = NonNegativeReals)


# Define equations for implicit variables

# Define Markdown Percent
def Define_Markdown_Percent_Rule(m,i):
    return m.MarkPct[i] == sum(m.SelectPrice[i,p]*m.mrkPriceGrid[i,p] for p in m.PGRID)
m.Define_Markdown_Percent =  Constraint(m.ITEMS, rule = Define_Markdown_Percent_Rule)

# Define Markdown Price
def Define_MarkPrice_Rule(m,i):
    return (1-m.MarkPct[i])*m.basePrice[i] == m.MarkPrice[i]
m.Define_MarkPrice =  Constraint(m.ITEMS, rule = Define_MarkPrice_Rule)

# Define Demand
def Define_Demand_Rule(m,i):
    return m.Demand[i] == sum(m.SelectPrice[i,p]*m.demandPriceGrid[i,p] for p in m.PGRID)
m.Define_Demand =  Constraint(m.ITEMS, rule = Define_Demand_Rule)

# Define Revenue
def Define_Revenue_Rule(m,i):
    return m.Revenue[i] == sum(m.SelectPrice[i,p]*m.revenuePriceGrid[i,p] for p in m.PGRID)
m.Define_Revenue =  Constraint(m.ITEMS, rule = Define_Revenue_Rule)

# Define Final Inv
def Define_FinalInv_Rule(m,i):
    return  m.initInv[i] - m.Demand[i] == m.FinalInv[i]
m.Define_FinalInv =  Constraint(m.ITEMS, rule = Define_FinalInv_Rule)


# Define Constraints

# Only choose one price point per item
def Only_One_Price_Point_Item_Rule(m,i):
    return sum(m.SelectPrice[i,p] for p in m.PGRID) <= 1
m.Only_One_Price_Point_Item =  Constraint(m.ITEMS, rule = Only_One_Price_Point_Item_Rule)


# Only allow two items to be promoted
def Only_Two_Items_To_Promote_Rule(m):
    return sum(m.SelectPrice[i,p] for i in m.ITEMS for p in m.PGRID) <= 2
m.Only_Two_Items_To_Promote =  Constraint(rule = Only_Two_Items_To_Promote_Rule)


# Define max discount
def Max_Discount_Rule(m,i):
    return m.MarkPct[i] <= m.maxPct
m.Max_Discount = Constraint(m.ITEMS, rule = Max_Discount_Rule)  

# Sell such that left inventory is less than min Inv
def Leftover_Inv_Rule(m,i):
     return m.FinalInv[i] <= m.cap
m.Leftover_Inv_Rule = Constraint(m.ITEMS, rule = Leftover_Inv_Rule)


# Define Objective Function
def Max_Revenue_Rule(m):
    return sum((m.Revenue[i] - m.dispCost*m.FinalInv[i]) for i in m.ITEMS) 
m.Max_Revenue = Objective(rule = Max_Revenue_Rule, sense = maximize)

In [26]:
# Create data for the model
price_grid= pd.read_csv("../data/price_grid.csv")
instanceData = {None:{
    'PGRID': {None: price_grid['PRICE_ID'].unique()},
    'ITEMS': {None: price_grid['ITEM'].unique()},
    'mrkPriceGrid': price_grid.set_index(['ITEM','PRICE_ID']).to_dict()['MarkdownPct'],
    'demandPriceGrid': price_grid.set_index(['ITEM','PRICE_ID']).to_dict()['DEMAND'],
    'revenuePriceGrid': price_grid.set_index(['ITEM','PRICE_ID']).to_dict()['REVENUE']}}

# Instantiate model
instance = m.create_instance(instanceData)


### Solve

In [27]:
solver = SolverFactory('glpk')
solution = solver.solve(instance)

Check the status of the solver

In [28]:
print(solution['Solver'])


- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.053878068923950195



### Extract Optimum Values

Print optimum value of objective function

In [29]:
print(instance.Max_Revenue.expr())

439.095


Print optimum value of decision and implicit variables - Option 1 (tailored syntax, need to substitute the right indexes and variable names for other use cases)

In [30]:
data = {'ITEM': [], 'Opt_Discount': [], 'Opt_Price': [], 'Opt_Revenue': [], 'Final_Inv': []}
for i in instance.ITEMS:
    data['ITEM'].append(i)
    data['Opt_Discount'].append(instance.MarkPct[i].value)
    data['Opt_Price'].append(instance.MarkPrice[i].value)
    data['Opt_Revenue'].append(instance.Revenue[i].value)
    data['Final_Inv'].append(instance.FinalInv[i].value)
pd.DataFrame(data)

,ITEM,Opt_Discount,Opt_Price,Opt_Revenue,Final_Inv
0,A,0.0,21.50,0.000,10.0
1,B,0.5,14.75,221.250,15.0
2,C,0.7,9.15,262.605,12.3


Print optimum value of decision and implicit variables - Option 2 (generic, same syntax will work for other use cases)

In [33]:
data = {'Variable': [], 'Value': []}
for var in instance.component_objects(Var):
    for index in var:
        data['Variable'].append(f'{var.name}[{index}]')
        data['Value'].append(var[index].value)
pd.DataFrame(data)

,Variable,Value
0,"SelectPrice[('A', 1)]",0.000
1,"SelectPrice[('A', 2)]",0.000
2,"SelectPrice[('A', 3)]",0.000
3,"SelectPrice[('A', 4)]",0.000
4,"SelectPrice[('B', 1)]",0.000
5,"SelectPrice[('B', 2)]",0.000
6,"SelectPrice[('B', 3)]",1.000
7,"SelectPrice[('B', 4)]",0.000
8,"SelectPrice[('C', 1)]",0.000
9,"SelectPrice[('C', 2)]",0.000


In [12]:
price_grid.set_index(['PRICE_ID']).to_dict()['MarkdownPct']

{1: 0.1, 2: 0.3, 3: 0.5, 4: 0.7}